In [16]:
import fiftyone as fo
from fiftyone.utils import video
import os
import sys
from importlib import reload
import fiftyone.zoo as foz
from fiftyone import ViewField as F


Converting the avi video to mp4 is necessary

In [23]:
video.reencode_video('../datasets/AICity_data_S05_C010/validation/S05/c010/vdo.avi', '../datasets/AICity_data_S05_C010/validation/S05/c010/vdo.mp4')

In [2]:
# Create a dataset from a list of videos
dataset = fo.Dataset.from_videos(
    ['../datasets/AICity_data_S05_C010/validation/S05/c010/vdo.mp4']
)
# session = fo.launch_app(dataset)

 100% |█████████████████████| 1/1 [26.9ms elapsed, 0s remaining, 39.1 samples/s] 


We perform inference with Faster-RCNN on the video frames

In [5]:
model = foz.load_zoo_model("faster-rcnn-resnet50-fpn-coco-torch")
dataset.apply_model(model, "predictions")

 100% |██████|    1.2Gb/1.2Gb [3.2s elapsed, 0s remaining, 415.2Mb/s]       


c:\Users\Marcos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Marcos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\Marcos/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:02<00:00, 66.4MB/s] 


Computing metadata...
 100% |█████████████████████| 1/1 [3.4s elapsed, 0s remaining, 0.3 samples/s] 
 100% |███████████████| 4072/4072 [8.1m elapsed, 0s remaining, 8.3 samples/s]      


We only want the frames labeled by "car"

In [27]:
new_view = dataset.filter_labels("frames.predictions", F("label")=='car')

Then we export the model predictions that will act as initial groundtruth in order to create the annotations

In [30]:
new_view.export(
    export_dir="./outputs",
    dataset_type=fo.types.CVATVideoDataset,
    frame_labels_field='frames.predictions',
)

Directory './outputs' already exists; export will be merged with existing files
 100% |█████████████████████| 1/1 [15.3s elapsed, 0s remaining, 0.1 samples/s] 


Finally, we show some of those predictions

In [22]:
session = fo.launch_app(new_view)